In [1]:
# import all modules
# Third-party
import numpy as np
import torch

# Local files
import utilities
import train_NN as train
import neural_network as net

# Choose a device to run on - either CPU or GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [2]:
# Choose a ground truth PDE to analyze
a = 1 # PDE parameter

# True PDE - Network will satify this PDE at the end of training
def pde_fn(x, a):
    # PDE: u_xx = -(pi*a)^2.sin(pi*a*x)
    u_xx = -(np.pi * a)**2 * np.sin(np.pi * a * x)
    return u_xx

# True solution - Network will try to learn this
def u(x, a):
    return np.sin(np.pi * a * x)

In [3]:
# Create a dataset by sampling points from the PDE's domain
    #1. For the boundaries - subscript u
    #2. Inside the domain = residual - subscript r
# Here, 1D domain, so boundary is just two points! 
bc1_coords = np.array([[0.0], 
                       [0.0]]) 

bc2_coords = np.array([[1.0], 
                       [1.0]])

dom_coords = np.array([[0.0],
                       [1.0]])

no_of_data_samples  = 100
X_bc1 = dom_coords[0, 0] * np.ones((no_of_data_samples // 2, 1))
X_bc2 = dom_coords[1, 0] * np.ones((no_of_data_samples // 2, 1))

X_u = np.vstack([X_bc1, X_bc2]) # data for BC
Y_u = u(X_u, a) # y data for BC

X_r = np.linspace(dom_coords[0, 0],
                  dom_coords[1, 0], no_of_data_samples)[:, None] # data for residual
Y_r = pde_fn(X_r, a) # y data for residual

# values for normalizing teh datasets
# Normalize the inputs - X_u and X_r - (n_data_points, dimension)
mean = X_r.mean(axis=0)  # (1, dim) values
std = X_r.std(axis=0)  # (1, dim) values   

In [4]:
# Define model and a random seed
seed = 1
max_iterations = 10
model = net.PINN(no_of_neurons=10, no_of_h_layers=2,
                 mean= mean.item(), std =std.item())
model.to(device)

# Train the model -> default SGD with full batch and exponential weight decay
save_data_location = './data_ntk/'
details = train.train_nn_model(model, train_data=(X_u, Y_u, X_r, Y_r),
                            no_iterations=max_iterations, device=device,
                            save_data_location=save_data_location,
                            save_data_frequency=1)
details.keys()

dict_keys(['Loss', 'L_b', 'L_r'])

In [5]:
# Calculate the eigenvalues
import pickle
# load the jacobian matrices from hard disk
J_u, J_r = pickle.load(open("{}/Ju_Jr_{}.p".format(save_data_location,0),
                            'rb'))
eigs_all, eigs_uu, eigs_rr = utilities.calculate_eigenvalues_from_j(J_u, J_r)

In [6]:
eigs_all

array([ 5.36875585e+03,  2.96742460e+03,  9.08511427e+02,  8.11010827e+02,
        5.71942482e+02,  3.79975280e+02,  2.40505960e+02,  1.22761870e+02,
        7.72130185e+01,  4.93262484e+01,  4.46758870e+01,  2.50234950e+01,
        1.30581704e+01,  3.49132415e+00,  6.62963738e-01,  1.80092410e-01,
        6.88293781e-02,  8.07757262e-03,  1.84633922e-03,  6.56872472e-04,
        1.25380885e-04,  3.71431863e-05,  5.64958313e-06,  1.32266146e-06,
        1.76314066e-07,  6.40149729e-08,  1.68706084e-08,  2.72821697e-09,
        6.18936867e-10,  1.96933599e-10,  3.17538221e-11,  7.06926796e-12,
        1.11197648e-12,  6.71847823e-13,  5.08998454e-13,  4.67269148e-13,
        4.50232024e-13,  3.86700885e-13,  3.48090028e-13,  3.16995082e-13,
        3.00538779e-13,  2.86280375e-13,  2.60826917e-13,  2.49917368e-13,
        2.30305474e-13,  2.29421753e-13,  2.22045479e-13,  2.07367040e-13,
        1.96511065e-13,  1.90842574e-13,  1.83030943e-13,  1.74215601e-13,
        1.69602384e-13,  

In [7]:
eigs_uu

array([ 8.06233583e+02,  6.15739143e+01,  4.73273591e-13,  2.96309610e-13,
        2.39236897e-13,  2.03208525e-13,  1.65666723e-13,  1.14057128e-13,
        1.01787612e-13,  8.75285346e-14,  8.10985613e-14,  7.41895226e-14,
        6.51242506e-14,  5.56326859e-14,  4.97271058e-14,  4.60365343e-14,
        4.27358727e-14,  3.51755552e-14,  3.17071187e-14,  3.02490170e-14,
        2.77523563e-14,  2.51878641e-14,  2.15694894e-14,  2.08475430e-14,
        1.99594819e-14,  1.91451484e-14,  1.74689383e-14,  1.68007285e-14,
        1.50461386e-14,  1.38787111e-14,  1.29205443e-14,  1.21304273e-14,
        1.15368369e-14,  1.04179539e-14,  9.91907749e-15,  9.33709318e-15,
        9.07035865e-15,  8.84115536e-15,  7.84943221e-15,  7.16537703e-15,
        6.44316617e-15,  6.09667144e-15,  5.94536659e-15,  5.70933489e-15,
        5.02630396e-15,  4.88567841e-15,  4.61741121e-15,  4.11198841e-15,
        3.90539575e-15,  3.54175659e-15,  3.35275292e-15,  3.20646162e-15,
        2.85775683e-15,  

In [8]:
eigs_rr

array([ 5.36577931e+03,  2.96494005e+03,  8.70500574e+02,  3.80230749e+02,
        2.35396688e+02,  1.29991047e+02,  4.48126678e+01,  2.55882363e+01,
        1.33118501e+01,  5.01264666e+00,  6.65764468e-01,  1.96500723e-01,
        7.02668809e-02,  1.03622377e-02,  1.90733936e-03,  7.51444902e-04,
        1.44634530e-04,  3.73150993e-05,  5.69297560e-06,  1.64378275e-06,
        1.78080378e-07,  7.88626126e-08,  1.69023877e-08,  3.48180067e-09,
        6.73050010e-10,  2.48860795e-10,  3.20247374e-11,  7.30988593e-12,
        1.17570090e-12,  6.45321951e-13,  4.90348921e-13,  3.96379594e-13,
        3.58828119e-13,  3.27748506e-13,  3.02528140e-13,  2.66322177e-13,
        2.41313220e-13,  2.38028055e-13,  2.19160732e-13,  2.01962757e-13,
        1.96216093e-13,  1.81985550e-13,  1.75717004e-13,  1.65216258e-13,
        1.52312710e-13,  1.47711363e-13,  1.36830531e-13,  1.31341303e-13,
        1.20813045e-13,  1.17581273e-13,  1.17081500e-13,  1.08971288e-13,
        1.01647717e-13,  